In [3]:
import sys
import os
os.environ['PROJ_DATA'] = "/pscratch/sd/p/plutzner/proj_data"
import xarray as xr
import torch
import torchinfo
import random
import numpy as np
import importlib as imp
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import cartopy.crs as ccrs
import json
import pickle
import gzip
#import matplotlib.colors as mcolorsxx

%load_ext autoreload
%autoreload 2
import utils
import utils.filemethods as filemethods
import databuilder.data_loader as data_loader
from databuilder.data_generator import multi_input_data_organizer
import databuilder.data_loader as data_loader
from trainer.trainer import Trainer
from model.build_model import TorchModel
from utils import utils
# import databuilder.nino_indices as nino_indices # CAUSES CELL TO HANG

print(f"python version = {sys.version}")
print(f"numpy version = {np.__version__}")
print(f"xarray version = {xr.__version__}")
print(f"pytorch version = {torch.__version__}")

# https://github.com/victoresque/pytorch-template/tree/master

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
python version = 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:08:06) [GCC 11.3.0]
numpy version = 1.26.4
xarray version = 2024.5.0
pytorch version = 2.0.0.post104


In [4]:
config = utils.get_config("exp002")
seed = config["seed_list"][0]

In [5]:
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True

Process + Pickle Inputs and Targets

In [6]:
exp001_d_train = multi_input_data_organizer(config)

Opening MJO PCs
Opening Nino34 Data
/pscratch/sd/p/plutzner/E3SM/bigdata/E3SMv2data/member0101/member0101.Nino34.daily.int.nc
Opening Target data


In [13]:
exp001_d_train["y"][40:]

<xarray.DataArray 'PRECT' (time: 60186)> Size: 481kB
array([-9.40626658e-10, -6.82375384e-10, -7.81122513e-10, ...,
        3.17060679e-09, -8.73099789e-10, -7.95082906e-10])
Coordinates:
  * time     (time) object 481kB 1850-02-10 00:00:00 ... 2015-01-01 00:00:00
    lon      float64 8B 122.5
    lat      float64 8B 47.5

In [ ]:
# s_dict_train, s_dict_val, s_dict_test = multi_input_data_organizer(config)

# s_dict_savename1 = '/pscratch/sd/p/plutzner/E3SM/bigdata/presaved/exp002_train.pkl'
# with gzip.open(s_dict_savename1, "wb") as fp:
#     pickle.dump(s_dict_train, fp)

# s_dict_savename2 = '/pscratch/sd/p/plutzner/E3SM/bigdata/presaved/exp002_val.pkl'
# with gzip.open(s_dict_savename2, "wb") as fp:
#     pickle.dump(s_dict_val, fp)

# s_dict_savename3 = '/pscratch/sd/p/plutzner/E3SM/bigdata/presaved/exp002_test.pkl'
# with gzip.open(s_dict_savename3, "wb") as fp:
#     pickle.dump(s_dict_test, fp)

Opening MJO PCs
Opening Nino34 Data
/pscratch/sd/p/plutzner/E3SM/bigdata/E3SMv2data/member0101/member0101.Nino34.daily.int.nc
Opening Target data


Retrieve Data: 


In [ ]:
# Setup the Data
trainset = data_loader.CustomData(config["data_loader"]["data_dir"] + "exp002_train.pkl")
valset = data_loader.CustomData(config["data_loader"]["data_dir"] + "exp002_val.pkl")

train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=config["data_loader"]["batch_size"],
    shuffle=True,
    drop_last=False,
)

val_loader = torch.utils.data.DataLoader(
    valset,
    batch_size=config["data_loader"]["batch_size"],
    shuffle=False,
    drop_last=False,
)

Model Setup

In [ ]:
# Setup the Model
model = TorchModel(
    config=config["arch"],
    target_mean=trainset.target.mean(axis=0),
    target_std=trainset.target.std(axis=0),
)
model.freeze_layers(freeze_id="tau")
optimizer = getattr(torch.optim, config["optimizer"]["type"])(
    model.parameters(), **config["optimizer"]["args"]
)
criterion = getattr(module_loss, config["criterion"])()
metric_funcs = [getattr(module_metric, met) for met in config["metrics"]]

# Build the trainer
device = utils.prepare_device(config["device"])
trainer = Trainer(
    model,
    criterion,
    metric_funcs,
    optimizer,
    max_epochs=config["trainer"]["max_epochs"],
    data_loader=train_loader,
    validation_data_loader=val_loader,
    device=device,
    config=config,
)
